# 🌌 Spectral Affinity Master v6.3
### 9-Stage Ultimate Suno Master Pipeline

| # | Stage | Tech | Effect |
|---|-------|------|--------|
| 1 | 🧹 Neural Clean | DeepFilterNet 3 | AI artifact removal |
| 2 | 🎛️ Spectral Shaper | STFT Stabilizer | Resonance / harshness control |
| 3 | 🌀 Phase Shaper | STFT Phase | Phase coherence (-1 tight ← 0 → +1 analog) |
| 4 | 🔀 Stereo Wider | M/S + Haas | Synthetic stereo for mono Suno tracks |
| 5 | 🔊 Mono-Bass | Linkwitz-Riley | Sub-bass phase → mono |
| 6 | 💥 Transient Punch | Envelope Mask | Restore attack dynamics |
| 7 | ✨ Spectre Restore | Harmonic Exciter | 48kHz high-end recovery |
| 8 | 🏝️ Affinity Grouping | MERT + K-Means | Neural semantic clustering |
| 9 | 🎚️ Mastering Match | Matchering | Reference loudness & tone |

### 1. 🛠️ Setup

In [ ]:
import os, warnings, json, torch, torchaudio, librosa, glob, shutil, re, gc, sys
import numpy as np
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from IPython.display import HTML, display, clear_output
import torchaudio.transforms as T
import torchaudio.functional as F
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans as skKMeans

# 🤫 SHUT UP NOISE: Suppress CUDA/TF/Librosa warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONWARNINGS'] = 'ignore'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpu_name = torch.cuda.get_device_name(0) if device=='cuda' else 'CPU'
display(HTML(f"""
<style>
    .status-card {{ background: #1a1a1a; border-radius: 8px; padding: 15px; color: #00ffcc; font-family: 'Consolas', monospace; border-left: 5px solid #00ffcc; margin: 10px 0; }}
</style>
<div class='status-card'>🔥 ENGINE READY: {gpu_name}</div>
"""))

try: from nnAudio.Spectrogram import CQT1992v2
except: 
    !pip install -q nnAudio transformers deepfilternet matchering
    from nnAudio.Spectrogram import CQT1992v2
from transformers import Wav2Vec2FeatureExtractor, AutoModel

HAS_CUML = False
try: from cuml.cluster import KMeans as cuKMeans; HAS_CUML = True
except: pass

from ultimate_pipeline import UltimateSunoMaster, MasteringEngine, clean_name


### 2. 🧠 Neural Analysis Engine

In [ ]:
class SpectralMasterEngine:
    def __init__(self, device='cuda', sr=24000, cache_file='spectral_master_cache.json'):
        self.device, self.sr, self.cache_file = device, sr, cache_file
        self.cache = json.load(open(cache_file)) if os.path.exists(cache_file) else {}
        self.cqt = CQT1992v2(sr=sr, n_bins=84, bins_per_octave=12, verbose=False).to(device)
        major = torch.tensor([6.35,2.23,3.48,2.33,4.38,4.09,2.52,5.19,2.39,3.66,2.29,2.88], device=device)
        minor = torch.tensor([6.33,2.68,3.52,5.38,2.60,3.53,2.54,4.75,3.98,2.69,3.34,3.17], device=device)
        self.profiles = torch.stack([torch.roll(major,i) for i in range(12)] + [torch.roll(minor,i) for i in range(12)]).t()
        self.proc = Wav2Vec2FeatureExtractor.from_pretrained('m-a-p/MERT-v1-95M', trust_remote_code=True)
        self.mert = AutoModel.from_pretrained('m-a-p/MERT-v1-95M', trust_remote_code=True).to(device).eval()

    def save_cache(self): json.dump(self.cache, open(self.cache_file, 'w'))

    def get_camelot(self, key, mode):
        cm = {('B','major'):'01B',('F#','major'):'02B',('C#','major'):'03B',('G#','major'):'04B',
              ('D#','major'):'05B',('A#','major'):'06B',('F','major'):'07B',('C','major'):'08B',
              ('G','major'):'09B',('D','major'):'10B',('A','major'):'11B',('E','major'):'12B',
              ('G#','minor'):'01A',('D#','minor'):'02A',('A#','minor'):'03A',('F','minor'):'04A',
              ('C','minor'):'05A',('G','minor'):'06A',('D','minor'):'07A',('A','minor'):'08A',
              ('E','minor'):'09A',('B','minor'):'10A',('F#','minor'):'11A',('C#','minor'):'12A'}
        return cm.get((key, mode.lower()), '00X')

    def process_batch(self, paths_batch):
        def load_one(p):
            try:
                w, s = torchaudio.load(p)
                if s != self.sr: w = T.Resample(s, self.sr)(w)
                w = w.mean(0)
                if w.shape[0] > self.sr*120: w = w[:self.sr*120] # Max 2min for analysis
                return w, len(w)/self.sr
            except: return None
        
        # CPU Parallel Load
        audios, durs, valid_p = [], [], []
        with ThreadPoolExecutor() as pl: 
            res = list(pl.map(load_one, paths_batch))
        for i, r in enumerate(res):
            if r: audios.append(r[0]); durs.append(r[1]); valid_p.append(paths_batch[i])
        
        if not audios: return []
        
        # GPU Batch analysis
        m_len = max([a.shape[0] for a in audios])
        t = torch.zeros(len(audios), m_len, device=self.device)
        for i, a in enumerate(audios): t[i, :a.shape[0]] = a.to(self.device)
        
        with torch.no_grad():
            spec = self.cqt(t)
            energy = spec.pow(2).mean(dim=(1,2)).cpu().numpy()
            chroma = spec.view(len(audios),7,12,-1).sum(dim=(1,3))
            chroma = chroma / (chroma.norm(dim=1,keepdim=True)+1e-6)
            best = torch.argmax(torch.matmul(chroma, self.profiles), dim=1).cpu().numpy()
            embs = []
            for i in range(len(audios)):
                sl = int(self.sr*15); s = audios[i][:sl].cpu().numpy() # First 15s for MERT embs
                iv = self.proc(s, sampling_rate=self.sr, return_tensors='pt').input_values.to(self.device)
                embs.append(self.mert(iv).last_hidden_state.mean(dim=1).squeeze().cpu().numpy().tolist())
        
        pc = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
        return [{'key':pc[best[i]%12], 'mode':'major' if best[i]<12 else 'minor', 
                 'energy':float(energy[i]), 'duration':float(durs[i]), 
                 'embedding':embs[i], 'path':valid_p[i], 'camelot': self.get_camelot(pc[best[i]%12], 'major' if best[i]<12 else 'minor'),
                 'bpm': 120.0} for i in range(len(audios))]


### 3. 💎 Ultimate Suno Master (7-Stage Restoration)

In [ ]:
class UltimateSunoMaster:
    def __init__(self, device='cuda', target_sr=48000, stages=None):
        self.device, self.target_sr = device, target_sr
        self.stages = stages or {'neural_clean':True,'spectral_shape':True,'phase_shape':True,'stereo_widen':True,'mono_bass':True,'transient_punch':True,'spectre_restore':True}
        self.dfn_available = False
        if self.stages.get('neural_clean') and HAS_DFN:
            try: self._dfn_model, self._df_state, _ = init_df(); self.dfn_available = True; print('  ✅ DFN3 loaded')
            except Exception as e: print(f'  ⚠️ DFN3: {e}')

    def _to_48k(self, wav, sr):
        return T.Resample(sr, self.target_sr).to(self.device)(wav) if sr != self.target_sr else wav

    # ── Stage 1: Neural Clean ──
    def neural_clean(self, wav):
        if not self.dfn_available: return wav
        try: return df_enhance(self._dfn_model, self._df_state, wav, atten_lim_db=6)
        except: return wav

    # ── Stage 2: Spectral Shaper (Stabilizer) ──
    def spectral_shape(self, wav, amount=60, speed=90, sensitivity=30, focus_low=200, focus_high=16000):
        sr, n, h = self.target_sr, 4096, 1024
        win = torch.hann_window(n).to(self.device)
        sens_db = 6.0 - (sensitivity/100.0*5.0); max_cut = (amount/100.0)*8.0; alpha = 0.05+(speed/100.0*0.45)
        chs = []
        for ch in range(wav.shape[0]):
            stft = torch.stft(wav[ch], n_fft=n, hop_length=h, window=win, return_complex=True)
            mag, phase = stft.abs(), stft.angle()
            env = torch.nn.functional.conv1d(mag.t().unsqueeze(1), torch.ones(1,1,31,device=self.device)/31, padding=15).squeeze(1).t()
            excess = torch.clamp(20*torch.log10(mag+1e-8) - 20*torch.log10(env+1e-8) - sens_db, min=0)
            gain = 10**(-torch.clamp(excess*0.8, max=max_cut)/20)
            freqs = torch.linspace(0, sr/2, mag.shape[0]).to(self.device)
            mask = ((freqs>=focus_low)&(freqs<=focus_high)).float().unsqueeze(1)
            gain = gain*mask + (1.0-mask)
            for t in range(1, gain.shape[1]): gain[:,t] = alpha*gain[:,t] + (1-alpha)*gain[:,t-1]
            chs.append(torch.istft((mag*gain)*torch.exp(1j*phase), n_fft=n, hop_length=h, window=win, length=wav.shape[-1]))
        return torch.stack(chs)

    # ── Stage 3: Phase Shaper (-1 linearize ← 0 → +1 analogize) ──
    def phase_shape(self, wav, control=-0.3, stereo_link=True):
        if abs(control) < 0.01: return wav
        sr, n, h = self.target_sr, 4096, 1024
        win = torch.hann_window(n).to(self.device)
        chs, ref_delta = [], None
        for ch in range(wav.shape[0]):
            stft = torch.stft(wav[ch], n_fft=n, hop_length=h, window=win, return_complex=True)
            mag, phase = stft.abs(), stft.angle()
            if control < 0:
                bi = torch.arange(phase.shape[0], dtype=torch.float32, device=self.device)
                fi = torch.arange(phase.shape[1], dtype=torch.float32, device=self.device)
                expected = phase[:,:1] + (2*np.pi*bi.unsqueeze(1)*h/n) * fi.unsqueeze(0)
                delta = torch.atan2(torch.sin(phase-expected), torch.cos(phase-expected))
                factor = 1.0 + control
                if stereo_link and ch == 0: ref_delta = delta.clone()
                if stereo_link and ch > 0 and ref_delta is not None:
                    new_phase = expected + ref_delta*factor + (delta - ref_delta)
                else:
                    new_phase = expected + delta*factor
            else:
                freqs = torch.linspace(0, sr/2, phase.shape[0], device=self.device)
                analog = 0.15*torch.exp(-((freqs-80)/60)**2) - 0.08*torch.exp(-((freqs-3000)/2000)**2) + 0.12*torch.sigmoid((freqs-8000)/2000)
                new_phase = phase + analog.unsqueeze(1)*control
            chs.append(torch.istft(mag*torch.exp(1j*new_phase), n_fft=n, hop_length=h, window=win, length=wav.shape[-1]))
        return torch.stack(chs)

    # ── Stage 4: Stereo Wider (M/S + Synthetic Side) ──
    def stereo_widen(self, wav, width=0.3, generate_side=True):
        if wav.shape[0] < 2: wav = wav.expand(2,-1).clone()
        sr, n, h = self.target_sr, 4096, 1024
        win = torch.hann_window(n).to(self.device)
        Ls = torch.stft(wav[0], n_fft=n, hop_length=h, window=win, return_complex=True)
        Rs = torch.stft(wav[1], n_fft=n, hop_length=h, window=win, return_complex=True)
        M, S = (Ls+Rs)/2, (Ls-Rs)/2
        freqs = torch.linspace(0, sr/2, M.shape[0]).to(self.device)
        # Generate synthetic side if track is nearly mono
        ratio = (S.abs().pow(2).mean() / (M.abs().pow(2).mean()+1e-8)).item()
        if generate_side and ratio < 0.05:
            po = torch.linspace(0, 0.8, M.shape[0]).to(self.device) * torch.sigmoid((freqs-300)/200)
            S = S + M.abs()*0.18*torch.exp(1j*(M.angle()+po.unsqueeze(1)))
        # Freq-dependent width curve (bass narrow, presence wide, ultra-high rolloff)
        wc = torch.ones_like(freqs)*width
        wc *= torch.sigmoid((freqs-200)/100)
        wc *= 1.0 + 0.5*torch.exp(-((freqs-5000)/3000)**2)
        wc *= 1.0 - 0.3*torch.sigmoid((freqs-14000)/2000)
        Sw = S*(1.0+wc.unsqueeze(1))
        Lo = torch.istft(M+Sw, n_fft=n, hop_length=h, window=win, length=wav.shape[-1])
        Ro = torch.istft(M-Sw, n_fft=n, hop_length=h, window=win, length=wav.shape[-1])
        result = torch.stack([Lo, Ro])
        pk = result.abs().max()
        return result*(0.98/pk) if pk > 0.98 else result

    # ── Stage 5: Mono-Bass ──
    def mono_bass(self, wav, cutoff=150):
        low = F.lowpass_biquad(F.lowpass_biquad(wav, self.target_sr, cutoff), self.target_sr, cutoff)
        high = wav - low
        if wav.shape[0] >= 2: low = low.mean(dim=0, keepdim=True).expand_as(low)
        return low + high

    # ── Stage 6: Transient Punch ──
    def transient_punch(self, wav, boost_db=4.0):
        sr = self.target_sr
        mono = wav.mean(dim=0) if wav.shape[0]>=2 else wav.squeeze(0)
        fl = int(sr*0.005); hp = fl//2
        if fl < 2: return wav
        p = torch.nn.functional.pad(mono, (fl//2, fl//2))
        e = p.unfold(0, fl, hp).pow(2).mean(dim=-1).sqrt()
        flux = torch.clamp(torch.diff(e, prepend=e[:1]), min=0)
        if flux.max() < 1e-8: return wav
        fn = flux/(flux.max()+1e-8); thr = fn.mean()+1.5*fn.std()
        mask = torch.clamp((fn-thr)/(1.0-thr+1e-8), 0, 1)
        gain = torch.nn.functional.interpolate(mask[None,None,:], size=wav.shape[-1], mode='linear', align_corners=False).squeeze()
        rl = max(int(sr*0.025), 4)
        k = torch.exp(-torch.arange(rl, device=self.device, dtype=torch.float32)/(rl/4))
        k = (k/k.sum())[None,None,:]
        gain = torch.nn.functional.conv1d(gain[None,None,:], k, padding=rl//2).squeeze()[:wav.shape[-1]]
        r = wav*(1.0+gain.unsqueeze(0)*(10**(boost_db/20)-1.0))
        pk = r.abs().max()
        return r*(0.98/pk) if pk > 0.98 else r

    # ── Stage 7: Spectre Restore ──
    def spectre_restore(self, wav):
        sr = self.target_sr
        stft = torch.stft(wav[0], n_fft=4096, hop_length=1024, window=torch.hann_window(4096).to(self.device), return_complex=True)
        mdb = 20*torch.log10(stft.abs().mean(dim=1)+1e-8)
        freqs = torch.linspace(0, sr/2, mdb.shape[0]).to(self.device)
        v = mdb > (mdb.max()-55)
        co = freqs[v][-1].item() if v.any() else 16000.0
        co = max(12000.0, min(co, 20000.0))
        if co > 19500: return wav
        e1 = F.highpass_biquad(torch.tanh(F.highpass_biquad(wav, sr, co*0.85)*1.8), sr, co*0.9)
        e2 = F.highpass_biquad(torch.tanh(F.highpass_biquad(wav, sr, co)*3.0), sr, co)
        y = wav + e1*0.07 + e2*0.12
        pk = y.abs().max()
        return y*(0.98/pk) if pk > 0.98 else y


class MasteringEngine:
    def __init__(self, ref=None):
        self.ref, self.available = ref, False
        if ref and os.path.exists(str(ref)) and HAS_MATCHERING:
            self.available = True; print(f'  🎚️ Mastering ready | Ref: {os.path.basename(ref)}')
        else: print('  ℹ️ Mastering disabled' if not ref else f'  ⚠️ Ref not found: {ref}')
    def master(self, inp, out):
        if not self.available: shutil.copy2(inp,out) if inp!=out else None; return False
        try: mg.process(target=inp, reference=self.ref, results=[mg.pcm16(out)]); return True
        except Exception as e: print(f'  ⚠️ {e}'); shutil.copy2(inp,out) if inp!=out else None; return False

### 4. 🌈 Chromatic Flow

In [ ]:
def get_next_harmonic(c):
    n, a = int(c[:2]), c[2]
    return [f'{str(n).zfill(2)}{a}', f'{str((n%12)+1).zfill(2)}{a}', f'{str(((n-2)%12)+1).zfill(2)}{a}', f"{str(n).zfill(2)}{'A' if a=='B' else 'B'}"]

def sequence_chromatic_set(tracks, target):
    if not tracks: return []
    pool = list(tracks); cur = pool.pop(0); ordered = [cur]; dur = cur['duration']
    while pool and dur < target:
        ck = get_next_harmonic(cur['camelot'])
        def sc(t):
            h = 1.0 if t['camelot'] in ck else (0.8 if t['camelot']==cur['camelot'] else 0.0)
            b = max(0, 1.0-(abs(t['bpm']-cur['bpm'])/40.0))
            s = np.dot(cur['embedding'],t['embedding'])/(np.linalg.norm(cur['embedding'])*np.linalg.norm(t['embedding'])+1e-9)
            return h*0.5+s*0.3+b*0.2
        pool.sort(key=sc, reverse=True); nxt = pool.pop(0); ordered.append(nxt); dur += nxt['duration']; cur = nxt
    return ordered

def clean_name(n): return re.sub(r'[\-\_\.]+?', ' ', re.sub(r'^[\w\-]+?-', '', os.path.basename(n).rsplit('.',1)[0])).strip()

### 5. 🚀 Execute Pipeline

In [ ]:
# === CONFIG ===
INPUT_DIR       = '/kaggle/input/datasets/danieldobles/slavic-songs'
REFERENCE_TRACK = '/kaggle/input/datasets/danieldobles/slavic-songs/REF.flac'
OUTPUT_DIR      = '/kaggle/working/master_organized'
TEMP_DIR        = '/kaggle/working/_temp_restore'
BATCH_SIZE, SET_DUR, N_CLUSTERS = 16, 75*60, 3

STAGES = {'neural_clean':True,'spectral_shape':True,'phase_shape':True,'stereo_widen':True,'mono_bass':True,'transient_punch':True,'spectre_restore':True,'matchering':True}
SHAPER = {'amount': 60, 'speed': 90, 'sensitivity': 30}
PHASE_CONTROL, STEREO_WIDTH = -0.3, 0.3

print('\n🔧 Warming Up Pipeline...')
analyzer = SpectralMasterEngine(device=device); restorer = UltimateSunoMaster(device=device, stages=STAGES); mastering = MasteringEngine(ref=REFERENCE_TRACK)

paths = sorted(set(sum([glob.glob(os.path.join(INPUT_DIR,'**',e), recursive=True) for e in ['*.mp3','*.wav','*.flac','*.m4a']], [])))
to_do = [p for p in paths if p not in analyzer.cache]
if to_do:
    print(f'🔍 Analyzing {len(to_do)} new tracks...')
    for i in range(0, len(to_do), BATCH_SIZE):
        for r in analyzer.process_batch(to_do[i:i+BATCH_SIZE]): analyzer.cache[r['path']]=r
        analyzer.save_cache()

library = [analyzer.cache[p] for p in paths if p in analyzer.cache]
X = normalize(np.array([t['embedding'] for t in library]))
labels = (cuKMeans(n_clusters=N_CLUSTERS) if HAS_CUML else skKMeans(n_clusters=N_CLUSTERS, n_init=10)).fit_predict(X)
clusters = {i: [library[j] for j,l in enumerate(labels) if l==i] for i in range(N_CLUSTERS)}

if os.path.exists(OUTPUT_DIR): shutil.rmtree(OUTPUT_DIR)
os.makedirs(TEMP_DIR, exist_ok=True)

# ── DASHBOARD LOGIC ──
dashboard_html = """
<style>
    .dash {{ background: #000; color: #fff; padding: 20px; border-radius: 12px; font-family: 'Segoe UI', sans-serif; border: 1px solid #333; }}
    .bar {{ height: 10px; background: #333; border-radius: 5px; margin: 10px 0; overflow: hidden; }}
    .fill {{ height: 100%; background: linear-gradient(90deg, #00f2fe, #4facfe); width: 0%; transition: width 0.3s; }}
    .stat {{ font-size: 13px; color: #aaa; margin: 5px 0; }}
    .err-box {{ background: #300; border: 1px solid #f33; padding: 10px; margin-top: 15px; border-radius: 6px; display: none; }}
    .err-item {{ font-size: 12px; border-bottom: 1px solid #500; padding: 4px 0; color: #ff9999; }}
</style>
<div class='dash' id='dash'>
    <h2 style='margin:0; color:#4facfe'>💎 SPECTRAL AFFINITY DASHBOARD</h2>
    <div class='stat' id='current-track'>Initializing engine...</div>
    <div class='bar'><div class='fill' id='prog'></div></div>
    <div style='display:flex; justify-content:space-between'>
        <span class='stat' id='count'>0/0</span>
        <span class='stat' id='perc'>0%</span>
    </div>
    <div class='err-box' id='err-box'>
        <div style='color:#f33; font-weight:bold; margin-bottom:5px'>🛸 INCIDENT REPORT (Errors detected)</div>
        <div id='err-list'></div>
    </div>
</div>
"""
display(HTML(dashboard_html))

def update_ui(idx, total, name, errors):
    p = int((idx/total)*100)
    js = f"""
    document.getElementById('prog').style.width = '{p}%';
    document.getElementById('current-track').innerText = 'Processing: {name}';
    document.getElementById('count').innerText = '{idx}/{total}';
    document.getElementById('perc').innerText = '{p}%';
    """
    if errors:
        items = "".join([f"<div class='err-item'>⚠️ Stage: {e['stage']} | File: {e['file']} | {e['msg']}</div>" for e in errors[-5:]])
        js += f"document.getElementById('err-box').style.display = 'block';"
        js += f"document.getElementById('err-list').innerHTML = `{items}`;"
    display(HTML(f"<script>{js}</script>"))

# ── PROCESSOR ──
master_pool = ThreadPoolExecutor(max_workers=1)
futures, incident_logs = [], []
total_tracks = sum(len(c) for c in clusters.values())
processed_count = 0

for ci, ct in clusters.items():
    cn = f'Group_{chr(65+ci)}'; pool = sorted(ct, key=lambda x: x['energy']); si = 1
    while pool:
        oset = sequence_chromatic_set(pool, SET_DUR)
        if not oset: break
        pool = [t for t in pool if t['path'] not in {s['path'] for s in oset}]
        sd = os.path.join(OUTPUT_DIR, cn, f'Set_{si}'); os.makedirs(sd, exist_ok=True)
        for i, t in enumerate(oset):
            on = f"{str(i+1).zfill(2)} - [{t['camelot']}] {clean_name(t['path'])}.flac"
            fp, tp = os.path.join(sd, on), os.path.join(TEMP_DIR, f'tmp_{ci}_{si}_{i}.wav')
            
            res = restorer.process_track(t['path'], tp, shaper_params=SHAPER, phase_control=PHASE_CONTROL, stereo_width=STEREO_WIDTH)
            if res['status'] == 'ok':
                futures.append(master_pool.submit(lambda p1, p2: mastering.master(p1, p2) or os.remove(p1) if mastering.available else shutil.move(p1, p2), tp, fp))
            else:
                incident_logs.append({'file': clean_name(t['path']), 'stage': res['stage'], 'msg': res['msg']})
                shutil.copy2(t['path'], fp)
            
            processed_count += 1
            if processed_count % 2 == 0: update_ui(processed_count, total_tracks, clean_name(t['path']), incident_logs)
            if processed_count % 10 == 0: gc.collect(); torch.cuda.empty_cache()
        si += 1

update_ui(total_tracks, total_tracks, "Mastering Completed", incident_logs)
master_pool.shutdown(); shutil.rmtree(TEMP_DIR, ignore_errors=True)
!zip -0 -rq SlavMaster_v6_3.zip master_organized
display(HTML("<h3>🚀 <a href='SlavMaster_v6_3.zip' id='dl'>DOWNLOAD v6.3 MASTER</a></h3>"))
